# SMEFiT Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LHCfitNikhef/smefit_release/blob/tutorial/tutorial/tutorial.ipynb)

In [ ]:
# !pip install wget
# !pip install smefit
import sys
import os
import wget
import subprocess
import pathlib
import yaml
print("Python version:", sys.version)

In [ ]:
import smefit
import smefit.runner
from smefit.analyze import run_report
from smefit.analyze.report import Report

smefit.log.setup_console(None)

In [ ]:
def file_downloader(url, download_dir='./downloads'):

    if not os.path.exists(download_dir):
        os.mkdir(download_dir)

    file = wget.download(url, out=download_dir)
    return file

Download the smefit database

In [ ]:
smefit_database_url = 'https://github.com/LHCfitNikhef/smefit_database/archive/refs/heads/main.zip'
smefit_database = file_downloader(smefit_database_url);

Download the smefit runcards

In [ ]:
wget.download("https://raw.githubusercontent.com/LHCfitNikhef/smefit_release/tutorial/tutorial/runcard_fit.yaml", out="./downloads")
wget.download("https://raw.githubusercontent.com/LHCfitNikhef/smefit_release/tutorial/tutorial/runcard_report.yaml", out="./downloads")

In [ ]:
subprocess.run([f"unzip {smefit_database} -d ./downloads"], capture_output=False, shell=True, text=False)

EWPO

In [61]:
runner_ewpo = smefit.runner.Runner.from_file(pathlib.Path("./downloads/runcard_ttZ.yaml"))
runner_ewpo.global_analysis("A")

[18:39:11] INFO     Loading dataset : ATLAS_ttZ_13TeV                                                 ]8;id=659761;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=668244;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_ttZ_13TeV_2016                                            ]8;id=286973;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=993268;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_ttZ_13TeV_pTZ                                             ]8;id=640536;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=432903;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_ttZ_8TeV                                                  ]8;id=956512;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=541537;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : CMS_ttZ_13TeV                                                   ]8;id=34936;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=122719;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

...

           INFO     Loading dataset : CMS_ttZ_8TeV                                                    ]8;id=954766;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=20885;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

/Users/jaco/Documents/smefit_release/src/smefit/coefficients.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self._table.iloc[self.is_free, 0] = value


[18:39:12] INFO     Computing Analytic solution ...                                                 ]8;id=352848;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=117180;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py#127\127]8;;\

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Parameter ┃ Best value ┃ Error ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ Opt       │ 1.100      │ 9.133 │
│ OtG       │ -0.332     │ 1.703 │
└───────────┴────────────┴───────┘

           INFO     Sampling solutions ...                                                          ]8;id=205330;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=942575;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py#156\156]8;;\

In [62]:
def make_report(runcard_report):

    with open(runcard_report, encoding="utf-8") as f:
        report_config = yaml.safe_load(f)
    
    report_name = report_config["name"]
    report_path = pathlib.Path(report_config["report_path"]).absolute()
    report_folder = report_path.joinpath(f"{report_name}")

    report_folder.mkdir(exist_ok=True, parents=True)

    report = Report(report_path, report_config["result_path"], report_config)
    
    report.coefficients(**report_config["coefficients_plots"])

In [ ]:
make_report("./downloads/runcard_report_ewpo.yaml")

Make coefficient plots

Higgs

In [63]:
runner_higgs = smefit.runner.Runner.from_file(pathlib.Path("./downloads/runcard_higgs.yaml"))
runner_higgs.global_analysis("A")

[18:39:18] INFO     Loading dataset : ATLAS_CMS_SSinc_RunI                                            ]8;id=933730;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=386188;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_SSinc_RunII                                               ]8;id=659796;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=379121;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_STXS_runII_13TeV                                          ]8;id=901225;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=840413;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_WH_Hbb_13TeV                                              ]8;id=374845;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=132219;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_ZH_Hbb_13TeV                                              ]8;id=571082;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=843480;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

...

           INFO     Loading dataset : CMS_ggF_aa_13TeV                                                ]8;id=562259;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py\loader.py]8;;\:]8;id=213729;file:///Users/jaco/Documents/smefit_release/src/smefit/loader.py#135\135]8;;\

/Users/jaco/Documents/smefit_release/src/smefit/coefficients.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self._table.iloc[self.is_free, 0] = value


[18:39:19] INFO     Computing Analytic solution ...                                                 ]8;id=116358;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=789238;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py#127\127]8;;\

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Parameter ┃ Best value ┃ Error ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ Opt       │ -1.540     │ 2.864 │
│ OtG       │ 0.022      │ 0.026 │
└───────────┴────────────┴───────┘

           INFO     Sampling solutions ...                                                          ]8;id=221331;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=128808;file:///Users/jaco/Documents/smefit_release/src/smefit/optimize/analytic.py#156\156]8;;\

In [ ]:
make_report("./downloads/runcard_report_higgs.yaml")

EWPO + Higgs

In [ ]:
runner_higgs = smefit.runner.Runner.from_file(pathlib.Path("./downloads/runcard_ttZ_higgs.yaml"))
runner_higgs.global_analysis("A")

In [ ]:
make_report("./downloads/runcard_report_ttZ_higgs.yaml")